In [ ]:
################################################
# Copyright 2023 Benjamin M. Samudio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Ben Samudio, May 2023
# Towards Alleviating Suffering
###############################################

import pandas as pd
import tmap
from faerun import Faerun
from mhfp.encoder import MHFPEncoder

sb_ReLiEF_fingerprint_list = []
MinHash_permutations = 10000

sb_ReLiEF_fingerprint_file = 'sb-ReLiEF-Fingerprints_Abl-kinase-conformations.csv'
df = pd.read_csv(sb_ReLiEF_fingerprint_file)

for sb_ReLiEF_fingerprint_whole in df['Fingerprints']:
    sb_ReLiEF_fingerprint_split = sb_ReLiEF_fingerprint_whole.split(" ") 
    sb_ReLiEF_fingerprint_list.append(sb_ReLiEF_fingerprint_split)
    
MinHash_encoding = tmap.Minhash(MinHash_permutations)
LSH_encoding = tmap.LSHForest(MinHash_permutations)

LSH_encoding.batch_add(MinHash_encoding.batch_from_string_array(sb_ReLiEF_fingerprint_list))
LSH_encoding.index()

x, y, s, t, _ = tmap.layout_from_lsh_forest(LSH_encoding)

faerun = Faerun(clear_color = 'FFFFFF', view="free", coords=False, alpha_blending=True, coords_box=False, coords_grid=False, coords_color = '000000', coords_offset=2.0, coords_tick_count=10)
faerun.add_scatter(
    "Abl_kinase",
    {   "x": x, 
        "y": y,
        "c": df['Color'],
        "labels": df['Name']},
    point_scale=15,
    colormap = ['Set1'],
    has_legend=True,
    categorical=[True],
    series_title = ['Abl kinase conformations'],
    legend_labels = [None],
    fog_intensity = 0.5,
    shader = 'sphere'
)

faerun.add_tree("Abl_kinase_tree", {"from": s, "to": t}, point_helper="Abl_kinase")
faerun.plot("Abl_kinase",template="default", notebook_height=750)